## Imports

In [ ]:
import os
import sys
import datetime
dir_path = os.path.dirname(os.getcwd() + "/../src/")
sys.path.insert(1, dir_path)
import data_manip

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset as TorchDataset
import numpy as np
from sklearn import preprocessing
from scipy import stats
import matplotlib.pyplot as plt


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score

## Model Definitions

In [ ]:
def ids(mode=None, train_features=None, train_labels=None, test_features=None, test_labels=None):
    if mode == 'Forest':
        classifier = RandomForestClassifier(n_estimators = 10, random_state = 100)
        print("Classifier Initialised")
        classifier.fit(train_features, train_labels)
        print("Classifier Trained")
        predictions  = classifier.predict(test_features)
    elif mode == 'Logistic':
        classifier = LogisticRegression(random_state=100, max_iter=1000)
        print("Classifier Initialised")
        classifier.fit(train_features, train_labels)
        print("Classifier Trained")
        predictions = classifier.predict(test_features)
    elif mode == 'MLP':
        classifier = MLPClassifier(random_state=100, hidden_layer_sizes=(64,32), max_iter=1, activation='relu', solver='adam')
        print("Classifier Initialised")
        classifier.fit(train_features, train_labels)
        print("Classifier Trained")
        predictions = classifier.predict(test_features)
    else:
        print('Choose valid mode.')
    test_score = np.mean(test_labels == predictions)
    
    return predictions, test_score, classifier

In [ ]:
def basicFeatureImportance(df=None, classifier=None, name=None):
    if name == "Forest":
        num_feat = min(10,classifier.n_features_)
        importances = classifier.feature_importances_
        std = np.std([tree.feature_importances_ for tree in classifier.estimators_],
             axis=0)
        indices = np.argsort(importances)[::-1][:num_feat]

        print("Feature ranking:")

        for f in range(num_feat):
            print("feature ",indices[f],":", df.columns.values[indices[f]], importances[indices[f]])

    
        plt.figure()
        plt.title("Feature importances")
        plt.bar(range(num_feat), importances[indices],
            color="r", yerr=std[indices], align="center")
        plt.xticks(range(num_feat), indices)
        plt.xlim([-1, num_feat])
        plt.show()
        return df.columns.values[indices]
    if name == "Logistic":
        importance = classifier.coef_[0]
        # summarize feature importance
        for i,v in enumerate(importance):
            print('Feature: %0d, Score: %.5f' % (i,v))
        # plot feature importance
        plt.bar([x for x in range(len(importance))], importance)
        plt.show()

In [ ]:
def results(test_score=None, predictions=None, test_labels=None):
    print("Test Score: ", test_score)
    print(confusion_matrix(test_labels.values, predictions))
    
    print("F1 Score:", f1_score(test_labels.values, predictions, average='macro'))

    return f1_score(test_labels.values, predictions, average='macro')

## Test

It's not extremely clear how to recreate Sheatsley et al.'s methodology exactly, however, they explicitly state their criteria for considering a transformation to e valid. They constrain features to realisable values based on those that appear in the training data. We'll take their results as given and attempt to produce an attack that also fits this constraint.
Next, we load our dataset.

In [ ]:
unsw_metadata_file = "../metadata/unsw/metadata.json" # Put path to metadata file here
unsw_direc = None # Put path to the containing folder of UNSW NB15 CSV data here
if (unsw_metadata_file == None) or (unsw_direc == None):
    print("[*] Please provide a metadata file and/or path to UNSW NB15 directory")
metadata = data_manip.readMetadata(unsw_metadata_file)
df = data_manip.readDirec(unsw_direc, metadata)

In [ ]:
df.columns

### We see a marked difference in the ratio of TCP to UDP traffic in the Benign vs Malicious traffic

In [ ]:
ben_proto_value_counts = df[df[metadata["label_field"]] == metadata["benign_label"]]["proto"].value_counts()
ratio = ben_proto_value_counts["tcp"]/ben_proto_value_counts["udp"]
print(f"Ratio of TCP to UDP in Benign Traffic {ratio}")

In [ ]:
mal_proto_value_counts = df[df[metadata["label_field"]] != metadata["benign_label"]]["proto"].value_counts()
ratio = mal_proto_value_counts["tcp"]/mal_proto_value_counts["udp"]
print(f"Ratio of TCP to UDP in Malicious Traffic {ratio}")

### We see a similar difference looking at the ratio of traffic with Windows TTL to Linux TTLs in Benign vs Malicious Traffic

In [ ]:
top_benign_ttl = df[df[metadata["label_field"]] == metadata["benign_label"]][["sttl", "dttl"]].value_counts()[31, 29]
total_benign_length = df[df[metadata["label_field"]] == metadata["benign_label"]].shape[0]
print(f"Ratio of Top Windows TTLs to Other in Benign data: {top_benign_ttl/total_benign_length}")

In [ ]:
df[(df[metadata["label_field"]] != metadata["benign_label"]) & (df["synack"] == 0)][["proto", "synack", "ackdat"]].value_counts()

In [ ]:
df[(df[metadata["label_field"]] != metadata["benign_label"]) & (df["sttl"] == 254)]["proto"].value_counts().head(20)

In [ ]:
top_mal_ttl = df[df[metadata["label_field"]] != metadata["benign_label"]][["sttl", "dttl"]].value_counts()[254, 0]
total_mal_length = df[df[metadata["label_field"]] != metadata["benign_label"]].shape[0]
print(f"Ratio of Top Linux TTLs to Other in Mal data: {top_mal_ttl/total_mal_length}")

In [ ]:
def outlier_removal(train_features, train_labels, test_features, test_labels):
    """
    dloss and sloss have some pretty extreme outliers, more so than other features.
    """
    dloss_outliers = train_features["dloss"].quantile(0.97)
    sloss_outliers = train_features["sloss"].quantile(0.97)
    train_outliers = (train_features["dloss"] < dloss_outliers) & (train_features["sloss"] < sloss_outliers)
    train_features = train_features[train_outliers]
    train_labels = train_labels[train_outliers]
    test_outliers = (test_features["dloss"] < dloss_outliers) & (test_features["sloss"] < sloss_outliers)
    test_features = test_features[test_outliers]
    test_labels = test_labels[test_outliers]

    return train_features, train_labels, test_features, test_labels

## Define our simple perturbation test.

In [ ]:
class SimpleAdversarialTest():
    def __init__(self):
        pass

    def _importances(self, classifier):
        num_feat = min(10, classifier.n_features_in_)
        importances = classifier.feature_importances_
        std = np.std([tree.feature_importances_ for tree in classifier.estimators_],
            axis=0)
        indices = np.argsort(importances)[::-1][:num_feat]

        return importances, std, indices
    
    def _first_run(self, train_features, train_labels, test_features, test_labels):
        
        scaler = preprocessing.StandardScaler().fit(train_features)

        train_features = scaler.transform(train_features)
        test_features = scaler.transform(test_features)
        
        predictions, test_score, classifier = ids("MLP",
                                                    train_features=train_features,
                                                    train_labels=train_labels,
                                                    test_features=test_features,
                                                    test_labels=test_labels)
        results(test_score, predictions, test_labels)
        return classifier, train_features, train_labels, test_features, test_labels

    def _run(self, classifier, test_features):
        predictions = classifier.predict(test_features)
        return predictions

    def _adv(self, test_features, test_labels):
        labels = np.where(test_labels == 1)[0]
        test_features = test_features.to_numpy()
        # We have to adhere to the constraints set out by Sheatsley et al.
        # They define the concept of a 'Primary' feature which forces constraints upon all other features.
        # Selecting the Protocol as their primary feature, we can only choose values that appear with that feature.
        # So, if no flow exists with, say, Proto = UDP and STTL = 256, then perturbing the STTL of a UDP flow such that it equals 256 is forbidden.
        # Importantly, they do not consider the flow's label as a primary feature.
        # Between the benign and malicious traffic, all protocols have at least one flow with RTTs = 0 and TTLs = 0
        # Making these changes, we achieve an attack with perfect accuracy
        #test_features[labels, 0] = 1 # <-------- We can also change the protocol to UDP and still adhere to these constraints
        test_features[labels, 27] = 0 # ORIGINAL # Change RTT values 
        test_features[labels, 28] = 0 # ORIGINAL # Change RTT values
        test_features[labels, 29] = 0 # ORIGINAL # Change RTT values
        test_features[labels, 6] = 0 # ORIGINAL # dttl (Fixed by TCP/UDP as primary feature)
        test_features[labels, 5] = 0 # ORIGINAL # sttl (Fixed by TCP/UDP as primary feature)
        return test_features
    
    def pipeline(self, df, metadata, target_label):
        df, labels = data_manip.reformatForML(df, metadata, str(target_label))
        print(labels.value_counts())
        train_features, train_labels, test_features, test_labels, _, _ = data_manip.getTrainTestFeatures(df, labels)
        train_features, train_labels, test_features, test_labels = outlier_removal(train_features, train_labels, test_features, test_labels)
        classifier, _, _, _, _ = self._first_run(train_features, train_labels, test_features, test_labels)
        modified_test_features = self._adv(test_features, test_labels)
        scaler = preprocessing.StandardScaler().fit(train_features)
        modified_test_features = scaler.transform(modified_test_features)
        predictions = self._run(classifier, modified_test_features)
        test_score = np.mean(test_labels == predictions)
        misclassified = np.where((predictions == 1) & (test_labels == 1))
        results(test_score, predictions, test_labels)
        return test_features.iloc[misclassified]

In [ ]:
df.loc[df[metadata["label_field"]] != metadata["benign_label"], metadata["label_field"]] = "1"
print(df[metadata["label_field"]].value_counts())
test = SimpleAdversarialTest()
classify_correct = test.pipeline(df, metadata, "1")